#### From Pieter Abbeel's Lectures

##### Evaluating KL Divergence

$
\begin{align*}
KL(P(\tau; \theta) || P(\tau; \theta + \delta \theta)) &= \sum_{\tau} P(\tau; \theta) \log \dfrac{P(\tau; \theta)}{P(\tau; \theta + \delta \delta)} \\
&= \sum_{\tau} P(\tau; \theta) \log \dfrac{\sum_{t=0}^{H-1} \pi_{\theta} (a_t | s_t)}{\sum_{t=0}^{H-1} \pi_{\theta + \delta \theta} (a_t | s_t)} \\
& \approx \frac{1}{M} \sum_{\text{rollouts under }\theta} \log  \dfrac{\sum_{t=0}^{H-1} \pi_{\theta} (a_t | s_t)}{\sum_{t=0}^{H-1} \pi_{\theta + \delta \theta} (a_t | s_t)}\\
\end{align*}
$

Since dynamics cancels, and when we have expectation, we can use rollouts to estimate via Monte Carlo method

#### "Flat Tangent"

$KL(\pi_{\theta}(a|s) ||\pi_{\theta + \delta \theta}(a|s)) \approx \delta \theta^T \big( \sum_{(a,s) \sim \theta} \nabla_{\theta} \log \pi_{\theta}(a|s) \nabla_{\theta} \log \pi_{\theta}(a|s)^T \big) \delta \theta$

#### Studying

* Studying from https://www.telesens.co/2018/06/09/efficiently-computing-the-fisher-vector-product-in-trpo/

* The problem is that SGD offers no principled way to choose the right step size
* If the step size is too big, the optimization may miss the minimum
* If the step is too small, progress may be very slow
* Standard machine learning methods address this problem by using automatic learning rate adjustment such as the Adam optimizer

* TRPO reframes the optimization problem as a constrained optimization whose solution is guaranteed to result in an improved policy


#### The Problem

* The constrained optimization problem in TRPO

$\pi_{k+1} = argmax_{\pi} L(\pi)$

such that

$D_{KL}(\pi, \pi_{k}) \le \delta$

Here, $D_{KL}(\pi, \pi_{k})$ is defined as

$D_{KL}(\pi, \pi_{k}) = \sum (\pi_{k}) \log \dfrac{\pi_{k}}{\pi}$

$\pi_{k}$ refers to the output of the network at the $k^{th}$ iteration with parameters $\pi_{k}$. Since $\theta_{k}$ is fixed, the only variable in the formula is $\pi$

Therefore, when calculating derivatives using autograd, we must detach $\pi_{k}$ from the computation graph

#### Numerical Optimization

We use some numerical optimization techniques on (a) the loss function and (b) the constraint, specifically making Taylor series approximations to the functions

$L_{\theta_{k}}(\theta) = L_{\pi_{k}}(\pi) \sim L_{\theta_{k}}(\theta) + g^T (\theta - \theta_{k})$

$D_{KL}(\pi_{\theta_{k}}, \pi_{\theta}) = D_{KL}(\pi_{\theta_{k}},\pi_{\theta_{k}}) + \nabla_{\theta} D_{KL}(\pi_{\theta_{k}}, \pi_{\theta}) (\theta - \theta_{k}) + \frac{1}{2} (\theta - \theta_{k})^T H (\theta - \theta_{k})$

Our optimization problem reduces to 

$\theta_{k+1} = argmax_{\theta} g^T (\theta - \theta_{k})$

such that

$\frac{1}{2} (\theta - \theta_{k})^T H (\theta - \theta_{k}) \le \delta$

As shown in the appendix of the TRPO paper, this problem is solved in two steps--

1. A search direction for $\theta$ is computed 
2. A maximum distance along this direction is calculated such that the constraint is still satisfied

(1), the direction, can be calculated by applying the Lagrange multiplier technique.

Let $s = (\theta - \theta_{k})$. Then the Lagrangian is given by

$G = g^T s - \lambda \frac{1}{2} s^T H s$

Differentiation wrt $s$ (giving us our direction!) and setting to $0$, we get

$\frac{\partial G}{\partial s} = g - \lambda H s = 0$

The direction to search along is given by solving $Hs = g$. Now we must determine how far to move along this direction such that the constraint is still satisfied. Let this distance be denoted by $\beta$.

Thus, $\theta = \theta_{k} + \beta s$

Substituting this in the expression for the KL constraint, we get $\beta s^T H \beta s = \delta$, thus

$\beta = \sqrt{(\dfrac{2 \delta}{s^T H \beta s})}$

The product of $\beta$ and $s$ gives the optimal step to update $\theta$. This is the major contribution of TRPO, over ad hoc "learning rate schedule" typically used in training NNs.

1. Compute search direction by solving $Hs = g$
2. Maximum step size is computed using formula for $\beta$

Note that we are interested in the matrix-vector product $H^{-1}g$, not the matrix $H^{-1}$ by itself.

However calculating the Hessian matrix itself is a problem for autograd because its automatic differentiation feature is designed to calculate the derivative of a scalar wrt a vector, whereas the Hessian matrix involves the derivative of a vector (the derivative of the loss wrt the policy parameters) wrt a vector (policy parameters). One could of course loop over each element of the vector (code shown below), however this would be very slow, and require a lot of storage to store a K \times K Hessian matrix where K is a large number (in the thousands).

#### A Math Trick

Note

$H_{ij} = \dfrac{\partial}{\partial \theta_{j}} \dfrac{\partial D_{KL}}{\partial \theta_{i}}$

The $k^{th}$ element of the matrix vector product $Hx$ is:

$ \begin{align*}
y_{k} &= \sum_{j} H_{kj} x_{j} \\
&= \sum_{j} \dfrac{\partial}{\partial \theta_{j}} \dfrac{\partial D_{KL}}{\partial \theta_{k}} x_j \\
&= \dfrac{\partial}{\partial \theta_{k}} \sum_{j}  \dfrac{\partial D_{KL}}{\partial \theta_{k}} x_j  \\
\end{align*} $

The full vector is

$y = \dfrac{\partial}{\partial \theta} \sum_{j}  \dfrac{\partial D_{KL}}{\partial \theta_{k}} x_j $

Thus, the matrix-vector product can be calculated by first calculating the first derivative of the KL distance wrt the network params, and the product of this derivative vector with the input vector